In [ ]:
import pandas as pd
import numpy as np
import pathlib
import matplotlib.pyplot as plt
import sys
sys.path.append('ext')

In [ ]:
import plot_util

In [ ]:
inputdir = pathlib.Path.cwd() / 'inputdata' #/ 'Gforce'

In [ ]:
outputdir = pathlib.Path.cwd() / 'outputdata'
outputdir.mkdir(exist_ok=True)

In [ ]:
pathlib.Path.cwd()

In [ ]:
files = list(inputdir.glob('*/*/Accelerometer.csv'))
files

In [ ]:
# Start with first file to test
file = files[3]
data = pd.read_csv(file)
data.head()

In [ ]:
df = data.set_index('seconds_elapsed')[['x', 'y', 'z']]
df.head()

In [ ]:
len(df)

In [ ]:
df['norm'] = np.sqrt(df.apply(lambda x: x**2).sum(axis=1))
df.head()

In [ ]:
df.describe()

In [ ]:
df[['norm']].plot()

In [ ]:
# Double check units by comparing to uncalibrated -- if m/s^2 then should be offset by g

In [ ]:
file_unc = str(file).replace('Accelerometer', 'AccelerometerUncalibrated')
data_unc = pd.read_csv(file_unc)
data_unc.head()

In [ ]:
df_unc = data_unc.set_index('seconds_elapsed')[['x', 'y', 'z']]
df_unc['norm'] = np.sqrt(df_unc.apply(lambda x: x**2).sum(axis=1))
df_unc.describe()

In [ ]:
# Looks like uncalibrated is units of g

In [ ]:
# Make a function to clean file
def clean_df_from_accel_file(filepath):
    data = pd.read_csv(filepath)
    data = data.set_index('seconds_elapsed')[['x', 'y', 'z']]
    data['norm'] = np.sqrt(data.apply(lambda x: x**2).sum(axis=1))
    return data

In [ ]:
df = clean_df_from_accel_file(file)
df.head()

In [ ]:
df.describe()

In [ ]:
coords = ['x', 'y', 'z']

In [ ]:
ps_vals = df[coords].apply(lambda x: np.abs(np.fft.rfft(x))**2)
ps_vals.head()

In [ ]:
df.index.max()

In [ ]:
len(df) / df.index.max()

In [ ]:
df.index.min()

In [ ]:
len(df) / (df.index.max() - df.index.min())

In [ ]:
steps = df.index[1:] - df.index[:-1]
steps.max()

In [ ]:
steps.min()

In [ ]:
np.mean(steps)

In [ ]:
(steps > 0.011).sum()

In [ ]:
dt = np.median(steps)
dt

In [ ]:
plt.hist(dt, bins=np.arange(0.001, 0.03, 0.001))

In [ ]:
ps_vals['freq'] = np.fft.rfftfreq(len(df), d=dt)
ps = ps_vals.copy()
ps = ps.set_index('freq')
ps.head()

In [ ]:
np.log10(ps).plot()

In [ ]:
# Forgot to normalize by dt^2
ps *= dt**2

In [ ]:
np.log10(ps.sum(axis=1)).plot()

In [ ]:
# Units should be log10(W/kg/Hz)

In [ ]:
# Smooth
pss = pd.DataFrame(ps.sum(axis=1)).rename({0: 'Smoothed Power'}, axis=1)
pss['step10'] = np.floor(np.arange(len(pss)) / 100)
pss = pss.reset_index().groupby('step10').mean().set_index('freq')[['Smoothed Power']]
pss.head()

In [ ]:
# Test functionalized
# Smooth
def smooth(data, smooth_window=10):
    # Smooth is an int for the smoothing window
    if smooth_window <= 1:
        return data
    df = data.copy()
    df['step10'] = np.floor(np.arange(len(data)) / smooth_window)
    df = df.reset_index().groupby('step10').mean().set_index(data.index.name)#.drop('step10', axis=1)
    return df
pss = smooth(ps, 100)
pss.head()

In [ ]:
pss['Smoothed Power'] = pss.sum(axis=1)

In [ ]:
plot_util.generic_plot(np.log10(pss[['Smoothed Power']]), kind='line', xlabel='Hz', ylabel='log10(W/kg/Hz)?',
                       title=file.parent.stem, output_directory=outputdir)

In [ ]:
# Try log x-axis
plot_util.generic_plot(np.log10(pss).rename(np.log10, axis=0), kind='line', xlabel='Hz', ylabel='log10(W/kg/Hz)?',
                       title=str(file.parent.stem) + '_logged', output_directory=outputdir)